# Day 1

12/6/2019  
2:30-4:50 pm

In [32]:
# importing the requests library 
import requests 
import pandas as pd
import time

In [ ]:
# List of watersheds of interest
# The watershed identifiers, or hydrologic unit codes (HUC) of interest iclude:
HUC = ['11010001', '11010002',
       '11010003', '11010004',
      '11010005', '11010006',
      '11010007', '11010008',
      '11010009', '11010010',
      '11010011', '11010012'
      '11010013', '11010014'
      '08020301', '08020302'
      '08020303', '08020304']

In [262]:
# Main:

# api-endpoint 
# URL = "http://echo.epa.gov/tools/web-services/loading-tool/dmr_rest_services.get_dmr_loadings"
URL = 'https://ofmpub.epa.gov/echo/dmr_rest_services.get_dmr_loadings'  
# defining a params dict for the parameters to be sent to the API 
PARAMS = {'output':'JSON',
         'p_year':' 2019',
         'p_huc':'11010001',
#           'pageno' : 3,
         'responseset': 2000, # Number of rows to be retreived
         'download':'TopFacilityPounds' # This downloads each facility 5 times (1 per top pollutant)
         } # There might be a better API that does not do this repetititev.

  
# sending get request and saving the response as response object 
r = requests.get(url = URL, params = PARAMS) 
  
# extracting data in json format 
data = r.json() 

In [267]:

# api-endpoint 
# URL = "http://echo.epa.gov/tools/web-services/loading-tool/dmr_rest_services.get_dmr_loadings"
URL = 'https://ofmpub.epa.gov/echo/dmr_rest_services.get_all_dmr_facilities'  
# defining a params dict for the parameters to be sent to the API 
PARAMS = {'output':'JSON',
         'p_year':' 2019',
         'p_huc':'11010001',
#          'responseset': 6, # Number of rows to be retreived
#          'download':'TopFacilityPounds' # This downloads each facility 5 times (1 per top pollutant)
         } # There might be a better API that does not do this repetititev.

  
# sending get request and saving the response as response object 
r = requests.get(url = URL, params = PARAMS) 
  
# extracting data in json format 
data = r.json() 

In [268]:
data['Results'].keys()

dict_keys(['Message', 'QueryRows', 'PageCount', 'PageNo', 'StartRec', 'EndRec', 'Facilities'])

In [269]:
len(data['Results']['Facilities'])

500

In [264]:
len(data['Results']['TopFacilityPounds'])
fac_list=[]
for i in range(len(data['Results']['TopFacilityPounds'])):
    fac_list.append(data['Results']['TopFacilityPounds'][i]['ExternalPermitNmbr'])
    

In [251]:
fac_list

['MO0109096',
 'MO0109096',
 'MO0046931',
 'MO0046931',
 'AR0020010',
 'MO0113301',
 'MO0109096',
 'AR0021792',
 'MO0128376',
 'AR0022004',
 'MO0046931',
 'MO0046931',
 'MO0128376',
 'AR0020010',
 'MO0046931',
 'MO0109096',
 'AR0020010',
 'MOG130012',
 'AR0020010',
 'AR0020010',
 'MOG130012',
 'AR0022004',
 'AR0021741',
 'AR0022004',
 'AR0021741',
 'AR0049867',
 'MO0126900',
 'AR0022373',
 'MO0113247',
 'AR0022373',
 'MO0122866',
 'MO0129470',
 'AR0022373',
 'AR0022004',
 'MO0113247',
 'AR0021792',
 'MOG130012',
 'MO0046931',
 'MO0122866',
 'MO0113247',
 'MO0113247',
 'AR0021741',
 'AR0020010',
 'MO0122866',
 'AR0021792',
 'ARG640125',
 'AR0022004',
 'MO0046931',
 'MO0039578',
 'AR0047619',
 'MO0129470',
 'MO0039578',
 'AR0037249',
 'AR0021865',
 'AR0040118',
 'AR0049867',
 'AR0022373',
 'MO0039578',
 'AR0021865',
 'ARG640030',
 'AR0022004',
 'AR0021865',
 'MO0138266',
 'AR0037249',
 'AR0021741',
 'MO0122866',
 'MO0129470',
 'MO0123285',
 'MOG760164',
 'AR0021741',
 'MO0115096',
 'AR00

In [265]:
len(set(fac_list))

168

In [240]:
data['Results']['FacilityCounts']

{'AllFacs': '331',
 'AllMajor': '8',
 'AllMinor': '323',
 'FacInfoOnly': '163',
 'WithPermitData': '168',
 'WithPermitDataMajor': '4',
 'WithPermitDataMinor': '164',
 'WithMonReqOnly': '2',
 'WithMonReqLimits': '166',
 'WithDataIcis': '168'}

In [200]:
len(data['Results']['TopFacilityPounds'])

500

In [198]:
data['Results']['TopFacilityPounds'][:3]

[{'DataSource': 'ICIS',
  'ExternalPermitNmbr': 'MO0109096',
  'Uin': '110009712301',
  'FacilityName': 'FOREST PARK SUBDIVISION WWTP',
  'City': 'KIMBERLING CITY',
  'CountyName': 'STONE',
  'StateCode': 'MO',
  'Huc12': '110100011401',
  'Huc12Name': 'Cow Creek-Table Rock Lake',
  'SicCode1': '8641',
  'GeocodeLatitude': '36.617333',
  'GeocodeLongitude': '-93.442056',
  'PollutantDesc': 'Phosphorus',
  'SrsID': '153049',
  'ChemicalAbstractServiceNmbr': '7723140',
  'AverageDailyFlow': '1,250',
  'MajorMinorStatusFlag': 'Non-Major',
  'LoadOverLimit': '0',
  'LoadOverLimitTwpe': '--',
  'LimitsInfoFlag': None,
  'ExceedanceFlag': 'N',
  'QcFlag': 'Y',
  'TotalPounds': '19,707,540',
  'TotalTwpe': '0'},
 {'DataSource': 'ICIS',
  'ExternalPermitNmbr': 'MO0109096',
  'Uin': '110009712301',
  'FacilityName': 'FOREST PARK SUBDIVISION WWTP',
  'City': 'KIMBERLING CITY',
  'CountyName': 'STONE',
  'StateCode': 'MO',
  'Huc12': '110100011401',
  'Huc12Name': 'Cow Creek-Table Rock Lake',
  '

In [153]:
data['Results'].keys()

dict_keys(['Message', 'QueryID', 'FacilityCounts', 'QueryRows', 'PageCount', 'PageNo', 'StartRec', 'EndRec', 'TopFacilityPounds'])

In [154]:
data['Results']['QueryRows']

'1771'

In [266]:
data['Results']['FacilityCounts']

{'AllFacs': '331',
 'AllMajor': '8',
 'AllMinor': '323',
 'FacInfoOnly': '163',
 'WithPermitData': '168',
 'WithPermitDataMajor': '4',
 'WithPermitDataMinor': '164',
 'WithMonReqOnly': '2',
 'WithMonReqLimits': '166',
 'WithDataIcis': '168'}

In [156]:
data.keys()

dict_keys(['Results'])

In [157]:
data['Results']['TopFacilityPounds']

[{'DataSource': 'ICIS',
  'ExternalPermitNmbr': 'MO0113301',
  'Uin': '110009249107',
  'FacilityName': 'INDIAN POINT WATER ASSOCIATION WWTP NO 2',
  'City': 'BRANSON',
  'CountyName': 'STONE',
  'StateCode': 'MO',
  'Huc12': '110100011404',
  'Huc12Name': 'Table Rock Lake Dam-Table Rock Lake',
  'SicCode1': '8641',
  'GeocodeLatitude': '36.632278',
  'GeocodeLongitude': '-93.338583',
  'PollutantDesc': 'Ammonia as N',
  'SrsID': '152389',
  'ChemicalAbstractServiceNmbr': '7664417',
  'AverageDailyFlow': '765',
  'MajorMinorStatusFlag': 'Non-Major',
  'LoadOverLimit': '0',
  'LoadOverLimitTwpe': '0',
  'LimitsInfoFlag': 'Y',
  'ExceedanceFlag': 'N',
  'QcFlag': None,
  'TotalPounds': '5,502,582',
  'TotalTwpe': '6,108'},
 {'DataSource': 'ICIS',
  'ExternalPermitNmbr': 'MO0109096',
  'Uin': '110009712301',
  'FacilityName': 'FOREST PARK SUBDIVISION WWTP',
  'City': 'KIMBERLING CITY',
  'CountyName': 'STONE',
  'StateCode': 'MO',
  'Huc12': '110100011401',
  'Huc12Name': 'Cow Creek-Table

## Note: 

I always set to 1000 rows, but this can be replaced by a smarter approach, where you first query the number of all facilities, then based on that and at the end double check if all of them was retreived or not.

In [68]:
data['Results'].keys()

dict_keys(['Message', 'QueryID', 'FacilityCounts', 'QueryRows', 'PageCount', 'PageNo', 'StartRec', 'EndRec', 'TopFacilityPounds'])

In [71]:
data['Results']['FacilityCounts']

{'AllFacs': '331',
 'AllMajor': '8',
 'AllMinor': '323',
 'FacInfoOnly': '163',
 'WithPermitData': '168',
 'WithPermitDataMajor': '4',
 'WithPermitDataMinor': '164',
 'WithMonReqOnly': '2',
 'WithMonReqLimits': '166',
 'WithDataIcis': '168'}

In [160]:
facility_numbers = data['Results']['FacilityCounts']['AllFacs']
retrived_facility_numbers = len(data['Results']['TopFacilityPounds'])
print('Total number of unique facilities in this waterbody: {}'.format(facility_numbers))
print('Number of retirved facilities (non unique) in this call: {}'.format(retrived_facility_numbers))
if facility_numbers != retrived_facility_numbers:
    print('==='*30)
    print('WARNING: {} non-unique out of {} unique facilities were retrived!'.format(retrived_facility_numbers, facility_numbers))
    print('==='*30)

Total number of unique facilities in this waterbody: 746
Number of retirved facilities (non unique) in this call: 6


### Report:
1. It took a lot to find the correct URL for API call. Finally I found it from https://echo.epa.gov/tools/web-services/loading-tool#/Water%20Pollution%20Search%2C%20Facility%20Reports%2C%20and%20Loading%20Calculations/get_dmr_rest_services_get_dmr_loadings. If you select the Blue "Try it out" button, then scroll down, where you will see the corresponding URL under "Request URL" section.
2. Updated Jessie on the progress.
3. Next steps are:
    - work on different existing API calls to see which ones you need to extract the required information.

# ====================================
Day 2
12/12/2019 4:36 pm - 6:48 pm

Goal: 
- Look for the relevant API's that provides the data required by project.

In [161]:
facilities = data['Results']['TopFacilityPounds']

In [162]:
facilities_stats = data['Results']['TopFacilityPounds']

In [163]:
Success_flag = facility_counts = data['Results']['Message']

In [164]:
facilities

[{'DataSource': 'ICIS',
  'ExternalPermitNmbr': 'MO0113301',
  'Uin': '110009249107',
  'FacilityName': 'INDIAN POINT WATER ASSOCIATION WWTP NO 2',
  'City': 'BRANSON',
  'CountyName': 'STONE',
  'StateCode': 'MO',
  'Huc12': '110100011404',
  'Huc12Name': 'Table Rock Lake Dam-Table Rock Lake',
  'SicCode1': '8641',
  'GeocodeLatitude': '36.632278',
  'GeocodeLongitude': '-93.338583',
  'PollutantDesc': 'Ammonia as N',
  'SrsID': '152389',
  'ChemicalAbstractServiceNmbr': '7664417',
  'AverageDailyFlow': '765',
  'MajorMinorStatusFlag': 'Non-Major',
  'LoadOverLimit': '0',
  'LoadOverLimitTwpe': '0',
  'LimitsInfoFlag': 'Y',
  'ExceedanceFlag': 'N',
  'QcFlag': None,
  'TotalPounds': '5,502,582',
  'TotalTwpe': '6,108'},
 {'DataSource': 'ICIS',
  'ExternalPermitNmbr': 'MO0109096',
  'Uin': '110009712301',
  'FacilityName': 'FOREST PARK SUBDIVISION WWTP',
  'City': 'KIMBERLING CITY',
  'CountyName': 'STONE',
  'StateCode': 'MO',
  'Huc12': '110100011401',
  'Huc12Name': 'Cow Creek-Table

In [29]:
data_dict = {'ExternalPermitNmbr':[],
            'FacilityName':[],
            'City':[],
            'GeocodeLatitude':[],
            'GeocodeLongitude':[],
            'MajorMinorStatusFlag':[],
            'CountyName':[],
            'SicCode1':[],
             'zip':[],
             'FacilityType':[],
             'PermitType':[],
             'PermitEffectiveDate':[],
             'PermitExpirationDate':[],
             'ApprovedPretreatmentProgram':[],
             'FacilityDesignFlow':[],
             'AvgFacilityFlow':[],
             'SicCode':[],
             'Pollutants':[],
             'ParameterCode':[],
             'TotalPounds':[],
             'MaxAllowablePounds':[],
             'LoadOverLimit':[],
             'TotalTwpe':[],
             'MaxAllowableTwpe':[],
             'LoadOverLimitTwpe':[],
             'QcFlag':[],
             'ExceedanceCount':[],
             'QcReviewPounds':[],
             'QcReviewTwpe':[]
            }

facility_loading_URL = 'https://ofmpub.epa.gov/echo/dmr_rest_services.get_facility_report'  
    # defining a params dict for the parameters to be sent to the API 
    
facility_id_list = list() # just to check and eliminate the repeating facilities

for i in range(len(facilities)):
    if  (facilities[i]['ExternalPermitNmbr'] in facility_id_list)==False:
        facility_id_list.append(facilities[i]['ExternalPermitNmbr'])
        
        # only retrieve the data for facility if it is not retrieved before (based on the unique permit id number)
        PARAMS = {'output':'JSON',
                 'p_permit_id':facilities[i]['ExternalPermitNmbr'],
                 'p_year':'2019',
                 } 
        status_code = 0
        j=1
        # deal with unsuccessfull requests
        
        while status_code==0:
            try:
                # sending get request and saving the response as response object 
                facility_r = requests.get(url = facility_loading_URL, params = PARAMS) 
                status_code = 1
#                 print('Succ')
            except requests.exceptions.ConnectionError:
        # Does it ever gets here??????
                status_code = 0
                print('attempt {}, connection refused!'.format(j))
                j+=1
                facility_r.status_code='Connection refused'
        # extracting data in json format 
        facility_data = facility_r.json() 
        #         data_dict['Pollutants'].append(facility_data['Results']['PollutantLoads'])
        
        # Is there any better way than using for loop?
        for x in facility_data['Results']['PollutantLoads']:
            data_dict['ExternalPermitNmbr'].append(facilities[i]['ExternalPermitNmbr'])
            data_dict['FacilityName'].append(facilities[i]['FacilityName'])
            data_dict['City'].append(facilities[i]['City'])
            data_dict['GeocodeLatitude'].append(facilities[i]['GeocodeLatitude'])
            data_dict['GeocodeLongitude'].append(facilities[i]['GeocodeLongitude'])
            data_dict['MajorMinorStatusFlag'].append(facilities[i]['MajorMinorStatusFlag'])
            data_dict['CountyName'].append(facilities[i]['CountyName'])
            data_dict['SicCode1'].append(facilities[i]['SicCode1'])
            
            data_dict['zip'].append(facility_data['Results']['FacilityInfo']['FacZip'])
            data_dict['FacilityType'].append(facility_data['Results']['FacilityInfo']['FacilityType'])
            data_dict['PermitType'].append(facility_data['Results']['FacilityInfo']['PermitType'])
            data_dict['PermitEffectiveDate'].append(facility_data['Results']['FacilityInfo']['PermitEffectiveDate'])
            data_dict['PermitExpirationDate'].append(facility_data['Results']['FacilityInfo']['PermitExpirationDate'])
            data_dict['ApprovedPretreatmentProgram'].append(facility_data['Results']['FacilityInfo']['PretreatProgram'])
            data_dict['FacilityDesignFlow'].append(facility_data['Results']['FacilityInfo']['FacilityDesignFlow'])
            data_dict['AvgFacilityFlow'].append(facility_data['Results']['FacilityInfo']['AvgFacilityFlow'])
            data_dict['SicCode'].append(facility_data['Results']['FacilityInfo']['SicCode'])
         
            data_dict['Pollutants'].append(x['PollutantName'])
            data_dict['ParameterCode'].append(x['ParameterCode'])
            data_dict['TotalPounds'].append(x['TotalPounds'])
            data_dict['MaxAllowablePounds'].append(x['MaxAllowablePounds'])
            data_dict['LoadOverLimit'].append(x['LoadOverLimit'])
            data_dict['TotalTwpe'].append(x['TotalTwpe'])
            data_dict['MaxAllowableTwpe'].append(x['MaxAllowableTwpe'])
            data_dict['LoadOverLimitTwpe'].append(x['LoadOverLimitTwpe'])
            data_dict['QcFlag'].append(x['QcFlag'])
            data_dict['ExceedanceCount'].append(x['ExceedanceCount'])
            data_dict['QcReviewPounds'].append(x['QcReviewPounds'])
            data_dict['QcReviewTwpe'].append(x['QcReviewTwpe'])


        print('{}/{}, {},     facility name: {}'.format(i+1, len(facilities),
                                                        facility_data['Results']['Message'],
                                                        facilities[i]['FacilityName']))
        time.sleep(0.5) # pause between API calls
  

1/150, Success,     facility name: FOREST PARK SUBDIVISION WWTP
3/150, Success,     facility name: KIMBERLING CITY WWTP
5/150, Success,     facility name: FAYETTEVILLE - NOLAND WWTP
6/150, Success,     facility name: INDIAN POINT WATER ASSOCIATION WWTP NO 2
8/150, Success,     facility name: BERRYVILLE, CITY OF-WWTP
9/150, Success,     facility name: INDIAN POINT WATER ASSOCIATION WWTP NO 2
10/150, Success,     facility name: HUNTSVILLE WWTF
18/150, Success,     facility name: MDC ROARING RIVER STATE FISH HATCHERY
23/150, Success,     facility name: GREEN FOREST, CITY OF-WWTP
26/150, Success,     facility name: BEDFORD FALLS MOBILE HOME PARK
27/150, Success,     facility name: OZARK VILLA WWTF
28/150, Success,     facility name: WEST FORK, CITY OF
29/150, Success,     facility name: TCRSD BIG BEAR RESORT #2 WWTP
31/150, Success,     facility name: COMPTON RIDGE WWTF
32/150, Success,     facility name: BOX CANYON WATERSHED WWTP
46/150, Success,     facility name: BENTON/WASHINGTON REGIO

In [30]:
df=pd.DataFrame(data_dict)

In [31]:
df.head(20)

,ExternalPermitNmbr,FacilityName,City,GeocodeLatitude,GeocodeLongitude,MajorMinorStatusFlag,CountyName,SicCode1,zip,FacilityType,...,TotalPounds,MaxAllowablePounds,LoadOverLimit,TotalTwpe,MaxAllowableTwpe,LoadOverLimitTwpe,QcFlag,ExceedanceCount,QcReviewPounds,QcReviewTwpe
0,MO0109096,FOREST PARK SUBDIVISION WWTP,KIMBERLING CITY,36.617333,-93.442056,Non-Major,STONE,8641,65686,NON-POTW,...,"3,659,967","13,724,903",0,"4,063","15,235",0,Yes,0,"3,659,967","4,063"
1,MO0109096,FOREST PARK SUBDIVISION WWTP,KIMBERLING CITY,36.617333,-93.442056,Non-Major,STONE,8641,65686,NON-POTW,...,0,"17,276",0,0,"8,638",0,No,0,0,0
2,MO0109096,FOREST PARK SUBDIVISION WWTP,KIMBERLING CITY,36.617333,-93.442056,Non-Major,STONE,8641,65686,NON-POTW,...,"316,739","105,576,290",0,--,--,--,Yes,0,"316,739",--
3,MO0109096,FOREST PARK SUBDIVISION WWTP,KIMBERLING CITY,36.617333,-93.442056,Non-Major,STONE,8641,65686,NON-POTW,...,"19,707,540",0,0,--,--,--,Yes,0,"19,707,540",--
4,MO0109096,FOREST PARK SUBDIVISION WWTP,KIMBERLING CITY,36.617333,-93.442056,Non-Major,STONE,8641,65686,NON-POTW,...,"16,575,763","105,576,290",0,--,--,--,Yes,0,"16,575,763",--
5,MO0046931,KIMBERLING CITY WWTP,KIMBERLING CITY,36.627222,-93.423833,Non-Major,STONE,4952,65686-0370,POTW,...,"878,383",0,0,"52,703",0,0,Yes,0,"878,383","52,703"
6,MO0046931,KIMBERLING CITY WWTP,KIMBERLING CITY,36.627222,-93.423833,Non-Major,STONE,4952,65686-0370,POTW,...,"2,354,641","6,789,554","1,312","2,614","7,536",1.46,Yes,6,"2,354,641","2,614"
7,MO0046931,KIMBERLING CITY WWTP,KIMBERLING CITY,36.627222,-93.423833,Non-Major,STONE,4952,65686-0370,POTW,...,0,"12,433,294",0,0,--,--,Yes,0,0,0
8,MO0046931,KIMBERLING CITY WWTP,KIMBERLING CITY,36.627222,-93.423833,Non-Major,STONE,4952,65686-0370,POTW,...,"8,314,290","71,759,797",0,--,--,--,Yes,0,"8,314,290",--
9,MO0046931,KIMBERLING CITY WWTP,KIMBERLING CITY,36.627222,-93.423833,Non-Major,STONE,4952,65686-0370,POTW,...,"15,353,660","71,759,797",0,--,--,--,Yes,2,"15,353,660",--


### Achievements at the end of day 2:


1. Decoded the fields of json file
2. Retrived a lot of fields of data using API.
3. At the moment I am saving first as a dictionary, then convert it to a dataframe.


### Things to do on day 3:
1. Identify list of existing and missing data
2. Work on retriving the missing data, maybe using some extra API calls about the specific polutant or the specific facility
3. Code optimization, specially is there a better way than using dictionary+df?
4. Populate the year and ID number columns of df.
5. Combine the facility name with city name and figure out what is the trailing number? Is it post code?
6. Work on the name of columns of df to match it with column names in Database Template.

# Day 3
12/19/2019  

11:46 am - 12:40 pm

4:00 pm - 7:00 pm


List of missing columns:
- Facility type (example: POTW)
- Permit type (example: NPDES Individual Permit)
- Permit Effective Date
- Permit Expiration Date
- Approved Pretreatment Program
- Facility Design Flow (Permit Application) (MGD)
- Parameter code
- MAx Allowable load
- Max Allowable TWPE (lb-eq/yr)
- TWPE overlimit
- Contains potential Outliers?
- Number of Exceedances

I think majority of these missing columns relate to facility information. So lets query the information about each facility.

## Achievements on day 3: 

**Major progress**

- Retreived almost all of the required fields in the proper format:
    - There is multiple rows per facility corresponding to each pollutant
    
    
    
    


## Plan for Day 4:
 1. Look again to columns of the dataframe and look for missing fields
 2. If no missing field, then consider possible issues:
     - I am using try-except inside a while loop to catch the unresponsiveness from server. Not sure if it catches anything at all! Make sure about this.
     - The main (outer) query that retreives all facilities per watershed: I set the number of rows to 1000. It has a facility-stats that shows all facilities availablae in this watershed. Make sure at the end that the data for all of them are retreived.
 3. Next step will be to add another outer call that calls all the watersheds. (So far I ahve been calling only one watershed.). This can be done with for loop that or one call with multiple watershed codes in it. Think about this, which one is better.
 4. Finally you need to add another layer where you retreive data from different years. You can not provide multiple years within one API call. So this definitely needs separate calls for each year.

## Day 4:

9:15am - 1:18 pm


1. Added a block to check the number of retrived data againest the actual number of facilities per waterbody
2. Working on multiple waterbodies ...
  - Can I retrieve only the name or id of the facilities (unique) for each waterbody, not with accompanying information? I looked at different rest services offerred by ECHO and did not find a service that gives only the name of the facilities for a specific waterbody and a specific year. So I am going to use my old method. Maybe there is something that can be done that I did not find. This can be considered as one of the future updates for the code. Notice that I found a service that returns the name of all facilities for a specific waterbody, but there is no filtering based on the year, which is required in my current application. In other words I need to look at the facilities within a specific region and Year!





### Questions:
1. Is there a better way than using for loop to populate the dictionary with retrieved data?
2. 